In [1]:
import json
import sys
from pathlib import Path

# Add parent directory to path to import from implementation package
# Notebooks are in implementation/notebooks/, so we go up two levels to project root
sys.path.insert(0, str(Path().resolve().parent.parent))

from implementation.gather_data import fetch_and_create_imdb_movie

ImportError: cannot import name 'generate_plot_metadata' from 'implementation.llm_generations' (/Users/michaelkeohane/Documents/movie-finder-rag/implementation/llm_generations.py)

In [6]:
movies_to_upsert_to_db = [13397]

In [ ]:
# Fetch all movies from the list of TMDB IDs
fetched_movies = [fetch_and_create_imdb_movie(movie_id) for movie_id in movies_to_upsert_to_db]

# Load existing movies from saved_imdb_movies.json
json_path = Path("../saved_imdb_movies.json")
with open(json_path, "r", encoding="utf-8") as f:
    existing_movies = json.load(f)

# Create a dictionary keyed by tmdb_id for efficient lookup and updates
# This allows us to quickly find and replace movies with the same TMDB ID
movies_dict = {movie["tmdb_id"]: movie for movie in existing_movies}

# Upsert fetched movies: if a movie with the same TMDB ID exists, override it; otherwise add it
for result in fetched_movies:
    if result["success"] and result["imdb_movie"]:
        # Convert IMDBMovie object to dictionary for JSON serialization
        movie_dict = result["imdb_movie"].model_dump(mode='json')
        # Upsert: update existing entry or add new one
        movies_dict[movie_dict["tmdb_id"]] = movie_dict
    else:
        print(f"Failed to fetch movie: {result.get('failure_reason', 'Unknown error')}")

# Convert back to list and save to JSON file
updated_movies_list = list(movies_dict.values())
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(updated_movies_list, f, indent=2, ensure_ascii=False)

print(f"Successfully upserted {len(fetched_movies)} movies. Total movies in file: {len(updated_movies_list)}")

Successfully upserted 1 movies. Total movies in file: 50
